<a href="https://colab.research.google.com/github/Alry23/cuda-samples/blob/master/SPH_2D_elastic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Define the SPH class
class SPH:
    def __init__(self, mass, rho0, h, k, mu):
        self.mass = mass
        self.rho0 = rho0
        self.h = h
        self.k = k
        self.mu = mu

    def W(self, r):
        q = np.linalg.norm(r) / self.h
        if q <= 1:
            return (10 / (7 * np.pi * self.h ** 2)) * (1 - 1.5 * q ** 2 + 0.75 * q ** 3)
        elif q <= 2:
            return (10 / (7 * np.pi * self.h ** 2)) * (0.25 * (2 - q) ** 3)
        else:
            return 0

    def gradW(self, r):
        q = np.linalg.norm(r) / self.h
        if q <= 1:
            return (-30 / (7 * np.pi * self.h ** 3)) * r * (q - 2)
        elif q <= 2:
            return (-30 / (7 * np.pi * self.h ** 3)) * r * (-0.25 * (2 - q) ** 2)
        else:
            return np.zeros_like(r)

    def rho(self, r):
        return self.mass * self.W(r)

    def gradP(self, rho):
        return self.k * (rho - self.rho0)

    def laplV(self, v):
        return self.mu * v

# Define the simulation parameters
mass = 1
rho0 = 1000
h = 1
k = 1
mu = 1

# Create an instance of the SPH class
sph = SPH(mass, rho0, h, k, mu)

# Define the initial positions and velocities of the particles
x = np.array([[0, 0], [1, 0], [0, 1], [1, 1]], dtype=np.float64)
v = np.zeros_like(x)

# Set the time step size
dt = 0.1

# Time integration loop
for t in range(10):
    # Compute the density and pressure at each particle
    rho = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            if i != j:
                rho[i] += sph.rho(x[i] - x[j])
    P = sph.gradP(rho)

    # Compute the acceleration of each particle
    a = np.zeros_like(x)
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            if i != j:
                a[i] += -sph.mass * (P[i] / rho[i] ** 2 + P[j] / rho[j] ** 2) * sph.gradW(x[i] - x[j])
                a[i] += sph.laplV(v[i] - v[j]) * sph.gradW(x[i] - x[j])

    # Update the positions and velocities of the particles
    x += v * dt + 0.5 * a * dt ** 2
    v += a * dt

# Print the final positions of the particles
print(x)
